<a href="https://colab.research.google.com/github/Abhishek2104200/QML/blob/main/HCQNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Step 2: Import and load the dataset
import pandas as pd

# Load CSV from your provided path
df = pd.read_csv('/content/drive/MyDrive/sars_cov2_kmer_encoded.csv')

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (49979, 97)


/tmp/ipython-input-2-4101777875.py:5: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/sars_cov2_kmer_encoded.csv')


,Accession,Organism_Name,GenBank_RefSeq,Submitters,Organization,Org_location,Release_Date,Pangolin,PangoVersions,Surveillance_Sampling,...,kmer_TCG,kmer_TCT,kmer_TGA,kmer_TGC,kmer_TGG,kmer_TGT,kmer_TTA,kmer_TTC,kmer_TTG,kmer_TTT
0,OX474929.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Northumbria University, Newcastle upon Tyne, N...",NaN,2023-04-29,B,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,0.003789,0.017966,0.021146,0.018372,0.018609,0.028725,0.029334,0.017222,0.027439,0.033800
1,MZ185301.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Howard,D., Batra,D., Cook,P.W., Moser,K., Pask...","Centers for Disease Control and Prevention, Re...",USA,2021-05-13,B.1.526,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,True,...,0.003733,0.018028,0.020955,0.018297,0.018667,0.028724,0.029330,0.017221,0.027446,0.034039
2,OU389953.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,London,United Kingdom,2021-07-13,B.1.1.7,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,0.003941,0.018020,0.021345,0.018225,0.018184,0.028733,0.029103,0.017568,0.027132,0.032633
3,OX597302.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Public Health Wales Microbiology Cardiff, Univ...",NaN,2023-05-13,B.1.177.15,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,0.003725,0.018223,0.021109,0.018223,0.018525,0.028861,0.029364,0.017216,0.027452,0.033761
4,PP603966.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Amin,H., Wang,J., Su,M., Akther,S., Bologna,J....","NYC DOHMH, Bureau of the Public Health Laboratory",USA,2024-04-09,B.1.1.434,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,0.003732,0.018155,0.021080,0.018323,0.018626,0.028813,0.029384,0.017281,0.027501,0.033822


In [ ]:
# Step 3: Extract k-mer features and labels
# Select columns starting with 'kmer_'
kmer_columns = [col for col in df.columns if col.startswith('kmer_')]
X = df[kmer_columns].values

# Target label for classification (assuming 'target' has 5 variant classes encoded as 0–4)
y = df['target'].values

print("Feature shape:", X.shape)
print("Target shape:", y.shape)


Feature shape: (49979, 64)
Target shape: (49979,)


In [ ]:
# Step 4: Normalize k-mer features to [0, 1] range
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print("Scaled features shape:", X_scaled.shape)
print("Min:", X_scaled.min(), "| Max:", X_scaled.max())


Scaled features shape: (49979, 64)
Min: 0.0 | Max: 1.0000000000000009


In [ ]:
# Step 5: Project 64-dim features down to 8-dim using a Linear Layer
import torch
import torch.nn as nn

# Convert features to torch tensor
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# Define linear projection layer
linear_proj = nn.Linear(64, 8)
X_projected = linear_proj(X_tensor)

print("Projected feature shape:", X_projected.shape)


Projected feature shape: torch.Size([49979, 8])


In [ ]:
# Install PennyLane (if not already installed)
!pip install pennylane


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.3 MB/s eta 0:00:00


# **Designed quantum layer**

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define a single layer of the quantum circuit
def quantum_circuit(inputs, weights):
    # Angle encoding (2 features per qubit → 4x2 = 8 features)
    for i in range(n_qubits):
        qml.RY(inputs[i*2], wires=i)
        qml.RZ(inputs[i*2+1], wires=i)

    # Custom entanglement + trainable rotations
    for i in range(n_qubits):
        qml.RY(weights[i, 0], wires=i)
        qml.RZ(weights[i, 1], wires=i)

    # Ring entanglement
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1)%n_qubits])

    # Output: Expectation values from each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Create a QNode
weight_shape = (n_qubits, 2)
qnode = qml.QNode(quantum_circuit, dev, interface="torch", diff_method="backprop")

# Torch layer wrapper
from torch.nn import Module
class QuantumLayer(Module):
    def __init__(self):
        super().__init__()
        self.q_weights = nn.Parameter(torch.randn(weight_shape) * 0.01)

    def forward(self, x):
        outputs = []
        for i in range(x.shape[0]):
            q_out = qnode(x[i], self.q_weights)  # list of floats
            q_out_tensor = torch.tensor(q_out, dtype=torch.float32)
            outputs.append(q_out_tensor)
        return torch.stack(outputs)


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


# **HCQNN Model**

In [ ]:
#STEP 7
import torch.nn.functional as F

class HybridHCQNN(nn.Module):
    def __init__(self):
        super(HybridHCQNN, self).__init__()
        self.fc1 = nn.Linear(64, 8)           # 64 → 8 projection
        self.dropout = nn.Dropout(0.2)
        self.quantum = QuantumLayer()         # 8 → 4 via quantum circuit
        self.fc2 = nn.Linear(4, 5)            # 4 → 5 classes

    def forward(self, x):
        x = F.relu(self.fc1(x))               # Classical projection + ReLU
        x = self.dropout(x)
        x = self.quantum(x)                   # Quantum feature transformation
        x = self.fc2(x)                       # Output logits
        return F.log_softmax(x, dim=1)        # Log-Softmax for NLLLoss




In [ ]:
#Step 8

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)


In [ ]:
# Step 9: Train HCQNN
model = HybridHCQNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()  # Since we use log_softmax

# Training loop
n_epochs = 25  # Start small to test
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == y_batch).sum().item()

    accuracy = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {accuracy:.2f}%")


Epoch 1/25 - Loss: 2056.3369 - Accuracy: 19.07%
Epoch 2/25 - Loss: 2013.9654 - Accuracy: 19.58%
Epoch 3/25 - Loss: 2014.0795 - Accuracy: 19.67%
Epoch 4/25 - Loss: 2013.8932 - Accuracy: 19.00%


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        preds = output.argmax(dim=1)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

# Print accuracy and classification report
print("Test Accuracy:", accuracy_score(y_true, y_pred) * 100)
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred))


Test Accuracy: 23.359343737494996

Classification Report:

              precision    recall  f1-score   support

           0       0.23      0.83      0.36      1997
           1       0.24      0.34      0.28      2000
           2       0.00      0.00      0.00      2000
           3       0.00      0.00      0.00      2000
           4       0.00      0.00      0.00      1999

    accuracy                           0.23      9996
   macro avg       0.09      0.23      0.13      9996
weighted avg       0.09      0.23      0.13      9996



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
np.bincount(y_train)  # or y


array([7988, 8000, 8000, 8000, 7995])

# **GPU**

In [ ]:
#step 6
import pennylane as qml
from pennylane import numpy as np

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define a single layer of the quantum circuit
def quantum_circuit(inputs, weights):
    # Angle encoding (2 features per qubit → 4x2 = 8 features)
    for i in range(n_qubits):
        qml.RY(inputs[i*2], wires=i)
        qml.RZ(inputs[i*2+1], wires=i)

    # Custom entanglement + trainable rotations
    for i in range(n_qubits):
        qml.RY(weights[i, 0], wires=i)
        qml.RZ(weights[i, 1], wires=i)

    # Ring entanglement
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1)%n_qubits])

    # Output: Expectation values from each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Create a QNode
weight_shape = (n_qubits, 2)
qnode = qml.QNode(quantum_circuit, dev, interface="torch", diff_method="backprop")

# Torch layer wrapper
from torch.nn import Module
class QuantumLayer(Module):
    def __init__(self):
        super().__init__()
        self.q_weights = nn.Parameter(torch.randn(weight_shape) * 0.01)

    def forward(self, x):
      outputs = []
      for i in range(x.shape[0]):
        q_out = qnode(x[i], self.q_weights)  # list of floats
        q_out_tensor = torch.tensor(q_out, dtype=torch.float32).to(x.device)  # <- Fix here
        outputs.append(q_out_tensor)
      return torch.stack(outputs)




/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
#step 7
import torch.nn.functional as F

class HybridHCQNN(nn.Module):
    def __init__(self):
        super(HybridHCQNN, self).__init__()
        self.fc1 = nn.Linear(64, 8)             # 64 → 8
        self.dropout = nn.Dropout(0.2)
        self.quantum = QuantumLayer()           # 8 → 4 (quantum layer)
        self.fc_mid = nn.Linear(4, 16)          # 🔸 Added: 4 → 16 hidden layer
        self.fc_out = nn.Linear(16, 5)          # 🔸 Final: 16 → 5 output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))                 # Classical projection
        x = self.dropout(x)
        x = self.quantum(x)                     # Quantum circuit
        x = F.relu(self.fc_mid(x))              # 🔸 New hidden layer activation
        x = self.fc_out(x)                      # Output layer
        return F.log_softmax(x, dim=1)          # For NLLLoss


In [ ]:
#Step 8

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
#step 9

# Move model to device
model = HybridHCQNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Training loop
n_epochs = 15  # Increase since model is stronger now
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    for X_batch, y_batch in train_loader:
        # 🔁 Move data to GPU
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == y_batch).sum().item()

    accuracy = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {accuracy:.2f}%")


Epoch 1/15 - Loss: 2013.5141 - Accuracy: 19.78%
Epoch 2/15 - Loss: 2012.1339 - Accuracy: 20.14%


KeyboardInterrupt: 

# **adding meta data**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load your full dataset
df = pd.read_csv('/content/drive/MyDrive/sars_cov2_kmer_encoded.csv')

# Select metadata features
meta_features = df[['seq_length', 'Country']].copy()


/tmp/ipython-input-12-2653295975.py:5: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/sars_cov2_kmer_encoded.csv')


In [ ]:
# Label encode Country
le_country = LabelEncoder()
meta_features['Country_encoded'] = le_country.fit_transform(meta_features['Country'])


In [ ]:
from sklearn.preprocessing import MinMaxScaler

meta_scaler = MinMaxScaler()
meta_scaled = meta_scaler.fit_transform(meta_features[['seq_length', 'Country_encoded']])


In [ ]:
X_kmer = df[[col for col in df.columns if col.startswith('kmer_')]]


In [ ]:
import numpy as np

X_full = np.concatenate([X_kmer.values, meta_scaled], axis=1)
print("New feature shape:", X_full.shape)  # Should be (49979, 64 + 2)


New feature shape: (49979, 66)


In [ ]:
class HybridHCQNN(nn.Module):
    def __init__(self):
        super(HybridHCQNN, self).__init__()
        self.fc1 = nn.Linear(66, 32)          # 🔺 Less compression: 66 → 32
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(32, 8)           # 🔁 Intermediate projection
        self.quantum = QuantumLayer()         # Quantum: 8 → 4
        self.fc3 = nn.Linear(4, 16)
        self.out = nn.Linear(16, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))               # Add more capacity
        x = self.quantum(x)
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return F.log_softmax(x, dim=1)


In [ ]:
# Step 4: Normalize k-mer features to [0, 1] range
from sklearn.preprocessing import MinMaxScaler
import torch

# Normalize full feature set (k-mers + metadata)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_full)

# Convert to torch tensor
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

print("Scaled features shape:", X_scaled.shape)
print("Min:", X_scaled.min(), "| Max:", X_scaled.max())



Scaled features shape: (49979, 66)
Min: 0.0 | Max: 1.0000000000000009


In [ ]:
#Step 8

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
# Instead of using X_scaled from k-mers only,
# Use this X_scaled that includes metadata (66 features)
X_scaled = scaler.fit_transform(X_full)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Define loss with weights
criterion = nn.NLLLoss(weight=weights_tensor)

# Model and optimizer
model = HybridHCQNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 15
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == y_batch).sum().item()

    accuracy = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {accuracy:.2f}%")


Epoch 1/15 - Loss: 2012.7770 - Accuracy: 19.53%


KeyboardInterrupt: 

# **random approach**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_full)  # X_full = 64 k-mers + metadata


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class StrongBaseline(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(66, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )

    def forward(self, x):
        return F.log_softmax(self.net(x), dim=1)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Setup
model = StrongBaseline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss(weight=weights_tensor)

# Training loop
n_epochs = 25
for epoch in range(n_epochs):
    model.train()
    total_loss, correct = 0, 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (output.argmax(dim=1) == y_batch).sum().item()

    acc = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}%")


Epoch 1/25 - Loss: 1054.9213 - Accuracy: 69.33%
Epoch 2/25 - Loss: 509.4899 - Accuracy: 87.94%
Epoch 3/25 - Loss: 456.4855 - Accuracy: 89.04%
Epoch 4/25 - Loss: 442.3659 - Accuracy: 89.23%
Epoch 5/25 - Loss: 441.7536 - Accuracy: 89.15%
Epoch 6/25 - Loss: 424.4988 - Accuracy: 89.62%
Epoch 7/25 - Loss: 417.1703 - Accuracy: 89.57%
Epoch 8/25 - Loss: 404.2799 - Accuracy: 89.99%
Epoch 9/25 - Loss: 419.2059 - Accuracy: 89.37%
Epoch 10/25 - Loss: 420.5379 - Accuracy: 89.51%
Epoch 11/25 - Loss: 429.2466 - Accuracy: 89.14%
Epoch 12/25 - Loss: 409.6070 - Accuracy: 89.64%
Epoch 13/25 - Loss: 404.5738 - Accuracy: 89.84%
Epoch 14/25 - Loss: 407.5607 - Accuracy: 89.69%
Epoch 15/25 - Loss: 406.3185 - Accuracy: 89.74%
Epoch 16/25 - Loss: 407.5522 - Accuracy: 89.83%
Epoch 17/25 - Loss: 392.0214 - Accuracy: 90.08%
Epoch 18/25 - Loss: 396.1063 - Accuracy: 89.91%
Epoch 19/25 - Loss: 389.4389 - Accuracy: 90.21%
Epoch 20/25 - Loss: 395.5110 - Accuracy: 90.03%
Epoch 21/25 - Loss: 387.5520 - Accuracy: 90.17%


In [ ]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output = model(X_batch)
        preds = output.argmax(dim=1).cpu().numpy()
        labels = y_batch.numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

print("Classification Report:")
print(classification_report(all_labels, all_preds))


Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      1997
           1       0.94      0.99      0.96      2000
           2       0.97      0.98      0.98      2000
           3       0.90      0.93      0.91      2000
           4       0.92      0.82      0.87      1999

    accuracy                           0.92      9996
   macro avg       0.92      0.92      0.92      9996
weighted avg       0.92      0.92      0.92      9996



In [ ]:
class HybridQML(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(66, 8)               # Classical down-projection
        self.quantum = QuantumLayer()             # Quantum layer (input 8 → output 4)
        self.fc2 = nn.Linear(4, 64)               # Back to classical
        self.out = nn.Linear(64, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.quantum(x)                       # Insert QML here
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.out(x), dim=1)


In [ ]:
model = HybridQML().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss(weight=weights_tensor)  # use same class weights as before


In [ ]:
n_epochs = 10  # Start with 10 to test performance

for epoch in range(n_epochs):
    model.train()
    total_loss, correct = 0, 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (output.argmax(dim=1) == y_batch).sum().item()

    acc = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}%")


Epoch 1/10 - Loss: 2012.8813 - Accuracy: 20.41%
Epoch 2/10 - Loss: 2010.5443 - Accuracy: 21.06%


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-38-2345425316.py", line 11, in <cell line: 0>
    output = model(X_batch)
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-36-3436588643.py", line 11, in forward
    x = self.quantum(x)                       # Insert QML here
        ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*ar

TypeError: object of type 'NoneType' has no len()

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output = model(X_batch)
        preds = output.argmax(dim=1).cpu().numpy()
        labels = y_batch.numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(all_labels, all_preds))


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn

n_qubits = 4  # You can try 8 later
dev = qml.device("default.qubit", wires=n_qubits)

weight_shape = (n_qubits, 2)

# Define quantum circuit
def quantum_circuit(inputs, weights):
    # Angle encoding
    for i in range(n_qubits):
        qml.RY(inputs[i*2], wires=i)
        qml.RZ(inputs[i*2+1], wires=i)

    # Trainable rotations
    for i in range(n_qubits):
        qml.RY(weights[i, 0], wires=i)
        qml.RZ(weights[i, 1], wires=i)

    # Ring entanglement
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1) % n_qubits])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# QNode with PyTorch interface
qnode = qml.QNode(quantum_circuit, dev, interface="torch", diff_method="backprop")

# Quantum layer class
class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()
        # Better initialization: uniform in [0, π]
        init_weights = torch.rand(weight_shape) * np.pi
        self.q_weights = nn.Parameter(init_weights)

    def forward(self, x):
        # x: [batch_size, 8]
        return torch.stack([qnode(sample, self.q_weights) for sample in x])


In [ ]:
import torch.nn.functional as F

class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()
        init_weights = torch.rand(weight_shape) * np.pi
        self.q_weights = nn.Parameter(init_weights)

    def forward(self, x):
        outputs = []
        for sample in x:
            q_out = qnode(sample, self.q_weights)  # This returns a list
            q_out_tensor = torch.tensor(q_out, dtype=torch.float32, device=sample.device)
            outputs.append(q_out_tensor)
        return torch.stack(outputs)



In [ ]:
model = HybridQML().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss(weight=weights_tensor)  # Use same class weights


In [ ]:
n_epochs = 10  # Start with 10, we’ll adjust if needed

for epoch in range(n_epochs):
    model.train()
    total_loss, correct = 0, 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (output.argmax(dim=1) == y_batch).sum().item()

    acc = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}%")
